In [1]:
!pip install tensorflow==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.9 MB/s eta 0:00:00
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached gast-0.5.4-py3-none-any.wh

In [2]:
!pip install keras==2.15.0

In [3]:
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Dot, Lambda
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [4]:
def build_model(query_tower_input_dim, product_tower_input_dim, embedding_dim=16):
    # Input layers for tokenized sequences
    input_query = Input(shape=(query_tower_input_dim,), name='input_query')
    final_query_embedding = Dense(embedding_dim, activation='linear', name='embedding_layer_query')(input_query)
    normalized_query = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), name='normalize_query')(final_query_embedding)

    input_product = Input(shape=(product_tower_input_dim,), name='input_product')
    final_product_embedding = Dense(embedding_dim, activation='linear', name='embedding_layer_product')(input_product)
    normalized_product = Lambda(lambda x: tf.keras.backend.l2_normalize(x, axis=-1), name='normalize_product')(final_product_embedding)

    cosine_similarity = Dot(axes=1, normalize=True, name='cosine_similarity')([normalized_product, normalized_query])

    # Build the model
    model = Model(inputs=[input_query, input_product], outputs=cosine_similarity)

    # Define a model checkpoint callback to save the best model weights during training
    checkpoint_path = 'best_model_weights.keras'  # Specify the path to save the best model
    model_checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',  # Monitor validation loss
        verbose=1,
        save_best_only=True,  # Save only the best model
        mode='min'  # Minimize the validation loss
    )

    # Compile the model with 'binary_crossentropy' loss as a string
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return input_query, input_product, model, model_checkpoint

# Example usage
embedding_dim=16
query_tower_input_dim = 32
product_tower_input_dim = 160
input_query, input_product, model, model_checkpoint = build_model(query_tower_input_dim, product_tower_input_dim)
